In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
from copy import deepcopy

from camera_conventions.parser import parse_toml
Blender = parse_toml('../conventions/Blender.toml')
OpenGL = parse_toml('../conventions/OpenGL.toml')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from camera_conventions.geometry import Geometry

arr = np.array([    
    0.0, 0.2, 0.5,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.7,
    0.2, -0.1, 0.7,
    0.0, 0.2, 0.5, 
    -0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7,
    0.0, 0.2, 0.5,      
    -0.2, -0.1, 0.3,
    0.2, -0.1, 0.3,
    0.2, -0.1, 0.3, 
    0.2, -0.1, 0.7,
    -0.2, -0.1, 0.7,
    -0.2, -0.1, 0.3, 
    0.2, -0.1, 0.3,
    -0.2, -0.1, 0.7]).reshape(-1, 3)
colors = np.array([
    1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0,
                  0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0,
                  0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0,
                  1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
                  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0
]).reshape(-1, 3)
colors = (colors * 255).astype(np.uint8)
arr[:, 2] -= 0.5
verts = arr
faces = np.arange(0, verts.shape[0]).reshape(-1, 3)
geom = Geometry(convention=OpenGL, verts=verts, faces=faces, vertex_colors=colors)
geom.convert(Blender).to_json('../assets/geometries/pyramid_blender.json')

In [23]:
from camera_conventions.camera import Camera

"""
(((-0.7071067690849304, 3.0908619663705394e-08, 0.7071067690849304, 1.0),
        (0.7071067690849304, 3.0908619663705394e-08, 0.7071067690849304, 1.0),
        (0.0, 1.0, -4.371138828673793e-08, 0.0),
        (0.0, 0.0, 0.0, 1.0)))
"""
bTestCam = Camera(
    convention=Blender,
    fhat=50.0 / 36.0,
    T=np.array(
        [
            [-np.sqrt(2) / 2, 0, np.sqrt(2) / 2, 1.0],
            [np.sqrt(2) / 2, 0, np.sqrt(2) / 2, 1.0],
            [0, 1.0, 0, 0],
            [0, 0, 0, 1.0],
        ]
    ),
)
bTestCam.to_json('../assets/cameras/testCam_blender.json')
bTestCam.T

array([[-0.70710678,  0.        ,  0.70710678,  1.        ],
       [ 0.70710678,  0.        ,  0.70710678,  1.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
Rt, C = bTestCam.T[:3, :3], bTestCam.T[:3, 3]
blender_to_opengl = np.linalg.inv(Blender.world_transformation_matrix)
Rt_opengl = blender_to_opengl @ Rt
C_opengl = blender_to_opengl @ C
opengl_T = np.eye(4)
opengl_T[:3, :3] = Rt_opengl
opengl_T[:3, 3] = C_opengl
opengl_T = np.linalg.inv(opengl_T)
opengl_T

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
opengl_to_blender = np.eye(4)
opengl_to_blender[:3, :3] = np.linalg.inv(blender_to_opengl)
np.linalg.inv(bTestCam.T) @ opengl_to_blender

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
bTestCam.convert(OpenGL).T

array([[ 0.70710678,  0.        , -0.70710678,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [25]:
np.testing.assert_allclose(bTestCam.convert(OpenGL).convert(Blender).T, bTestCam.T)

In [26]:
bTestCam.convert(OpenGL).to_json('../assets/cameras/testCam.json')